In [3]:
import requests
import pandas as pd

In [4]:
# 데이터셋에 나타난 고유한 도로명 모음집 csv 불러오기
df = pd.read_csv('./csv_merged/unique_road_names.csv')
df

,도로명
0,NaN
1,12
2,13
3,19-30
4,20
...,...
5218,회원천남길 64-1
5219,회원천북길
5220,회원천북길 491
5221,회원천북길 545


In [5]:
# "경상남도 "라는 단어를 도로명 각 값앞에 붙이기
df['도로명'] = "경상남도 " + df['도로명'].astype(str)
df

,도로명
0,경상남도 nan
1,경상남도 12
2,경상남도 13
3,경상남도 19-30
4,경상남도 20
...,...
5218,경상남도 회원천남길 64-1
5219,경상남도 회원천북길
5220,경상남도 회원천북길 491
5221,경상남도 회원천북길 545


In [6]:
# REST API 키 입력
REST_API_KEY = "f306c7b5268f915ecb185b3a4e5bcbea"

# 요청 URL
url = "https://dapi.kakao.com/v2/local/search/address.json"

# 요청 헤더
headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
}

In [ ]:
# 추이를 추적할 행정동 리스트는 다음과 같음.
"""
# 주민등록인원 추이 추적이 되는 행정동 리스트.

의창구 : 동읍,북면,대산면,의창동,팔룡동,명곡동,봉림동
성산구 : 반송동,용지동,중앙동,상남동,사파동,가음정동,성주동,웅남동
마산합포구 : 구산면,진동면,진북면,진전면,현동,가포동,월영동,문화동,반월중앙동,완월동,자산동,교방동,오동동,합포동,산호동
마산회원구 : 내서읍,회원1동,회원2동,석전동,회성동,양덕1동,양덕2동,합성1동,합성2동,구암1동,구암2동,봉암동
진해구 : 충무동,여좌동,태백동,경화동,병암동,석동,이동,자은동,덕산동,풍호동,웅천동,웅동1동,웅동2동
"""

# 행정동 편입에 따른 이상치 ( 의창구 용지동 -> 성산구 용지동 ) 변화에 대응하기위해 편입 당월 데이터의 흐름을 고려하여 데이터 프레임 재편.
"""
의창구 동읍, 북면, 대산면, 의창동, 팔룡동, 명곡동, 봉림동은 ui-chang_df를 그대로 써도 됨.
성산구 반송동, 상남동, 사파동, 가음정동, 성주동, 웅남동은 seongsan_df를 그대로 써도 됨.
마산합포구 구산면, 진동면, 진북면, 진전면, 현동, 가포동, 월영동, 문화동, 완월동, 자산동,
            합포동, 산호동은 masanhappo_df를 그대로 써도 됨.
마산회원구 내서읍, 회원1동, 회원2동, 회성동, 양덕1동, 양덕2동, 합성1동, 합성2동, 구암1동,
            구암2동, 봉암동은 masanhwewon_df를 그대로 써도 됨.
진해구 여좌동, 태백동, 경화동, 병암동, 석동, 이동, 자은동, 덕산동, 풍호동, 웅천동, 웅동1동,
            웅동2동은 jinhae_df를 그대로 써도 됨.

"""

# 이게 필요한 작업인가??????????????????

In [ ]:
# 주소 데이터를 하나씩 처리
# 반복문을 통해 각 주소에 대해 API 요청
# 받아온 좌표 데이터를 데이터프레임에 추가
for index, row in df.iterrows():
    params = {
        "query": row['도로명']
    }

    # GET 요청
    response = requests.get(url, headers=headers, params=params)

    # 결과 출력
    if response.status_code == 200:
        data = response.json()
        print(data)
        if data['documents']:
            df.at[index, 'lng'] = data['documents'][0]['x']
            df.at[index, 'lat'] = data['documents'][0]['y']
        else:
            print(f"검색 결과 없음: {row['도로명']}")
    else:
        print(f"요청 실패: {response.status_code}")

{'documents': [{'address': None, 'address_name': '경남 창원시 성산구 가양로52번길', 'address_type': 'ROAD', 'road_address': {'address_name': '경남 창원시 성산구 가양로52번길', 'building_name': '', 'main_building_no': '', 'region_1depth_name': '경남', 'region_2depth_name': '창원시 성산구', 'region_3depth_name': '가음동', 'road_name': '가양로52번길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '128.701434677621', 'y': '35.2056112473538', 'zone_no': ''}, 'x': '128.701434677621', 'y': '35.2056112473538'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
{'documents': [{'address': None, 'address_name': '경남 창원시 성산구 가음로92번길', 'address_type': 'ROAD', 'road_address': {'address_name': '경남 창원시 성산구 가음로92번길', 'building_name': '', 'main_building_no': '', 'region_1depth_name': '경남', 'region_2depth_name': '창원시 성산구', 'region_3depth_name': '가음동', 'road_name': '가음로92번길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '128.701989536521', 'y': '35.2038448995433', 'zone_no': ''}, 'x': '128.701989536521', 'y': '35.2038448